In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
random_state = None
import numpy as np
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import re, string, unicodedata
import inflect
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.stem.porter import *
stemmer = PorterStemmer()
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('stopwords')
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords as sw
sww = sw.words()
import spacy

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#Load data

In [0]:
pip install pytreebank

  Created wheel for pytreebank: filename=pytreebank-0.2.7-cp36-none-any.whl size=37072 sha256=78f4dcb3b03522dd4ffefb07ae0f71d7d688d724dc1db3e6d7903a2134757ed0
  Stored in directory: /root/.cache/pip/wheels/e0/b6/91/e9edcdbf464f623628d5c3aa9de28888c726e270b9a29f2368
Successfully built pytreebank


In [0]:
import pytreebank
import sys

out_path = '/content/drive/My Drive/Anchor/set_2/sst_{}.txt'
dataset = pytreebank.load_sst('/content/drive/My Drive/Anchor/sst/raw_data')

# Store train, dev and test in separate files
for category in ['train', 'test', 'dev']:
    with open(out_path.format(category), 'w') as outfile:
        for item in dataset[category]:
            outfile.write("{}\t{}\n".format(
                item.to_labeled_lines()[0][0] + 1,
                item.to_labeled_lines()[0][1]
            ))

In [0]:
from sklearn.preprocessing import LabelEncoder

def load_data(path):
  X, y = [], []
  with open(path, 'r') as myfile:
    for line in myfile:
      label,text = line.split("\t")
      if len(text) == 0:
        continue
      X.append(text)
      y.append(label)
  return X,y

In [0]:
X1,y1 = load_data('/content/drive/My Drive/Anchor/set_2/sst_train.txt')
X2,y2 = load_data('/content/drive/My Drive/Anchor/set_2/sst_dev.txt')
X3,y3 = load_data('/content/drive/My Drive/Anchor/set_2/sst_test.txt')
y1 = np.array(y1)
y2 = np.array(y2)
y3 = np.array(y3)

In [0]:
print(len(X1),len(y1))
print(len(X2),len(y2))
print(len(X3),len(y3))

8544 8544
1101 1101
2210 2210


#Preprocessing

In [0]:
#create contraction dictonary
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", 
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", 
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", 
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", 
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", 
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", 
                    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", 
                    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", 
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", 
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", 
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", 
                    "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", 
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", 
                    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

new_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
##expand contractions
def expand_contractions(sentence, contraction_dict = contraction_dict):
  def replace(match):
    return contraction_dict[match.group(0)]
  return new_re.sub(replace, sentence)

In [0]:
def remove_html_tags(textz):
  ##remove html tags
  new_text = []
  new_text = [re.sub(r'<.*?>',' ', x) for x in textz]
  return new_text

def expand_contracts(textz):
  ##expand contractions 
  new_text = []
  for x in textz:
    ##replace all sentences with lowercase
    new = expand_contractions(x.lower())
    new_text.append(new)
  return new_text

In [0]:
def remove_nonascii(wordz):
  ##remove non ASCII characters
  new = []
  for w in wordz:
    word = unicodedata.normalize('NFKD', w).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    new.append(word)
  return new

def split(wordz):
  new = []
  for w in wordz:
    if '-' in w:
      w = w.split('-')
      new += w
    else:
      new.append(w)
  return new

def remove_punctuation(wordz):
  ##remove punctuations
  new = []
  for w in wordz:
    word = re.sub(r'[^\w\s]','',w)
    if word != ' ' and word != '':
      new.append(word)
  return new

def remove_noalpha(wordz):
  ##remove words if they are not alphabets
  new = []
  for w in wordz:
    if w.isalpha():
      new.append(w)
  return new

def remove_no(wordz):
  ##remove numbers
  new = []
  for w in wordz:
    if not any(c.isdigit() for c in w):
      new.append(w)
  return new

def remove_stopwords(wordz):
  ##remove stopwords
  new = []
  for w in wordz:
    if w not in sw.words('english'):
      new.append(w)
  return new

def stemming(wordz):
  ##stemming words 
  new = []
  for w in wordz:
    word = stemmer.stem(w)
    new.append(word)
  return new

In [0]:
def preprocessing(text):
  new = []
  text1 = remove_html_tags(text)
  text2 = expand_contracts(text1)
  for x in text2:
    word_tokens = word_tokenize(x) 
    words = remove_nonascii(word_tokens)
    words = split(words)
    words = remove_punctuation(words)
    #words = remove_no(words)
    words = remove_stopwords(words)
    words = remove_noalpha(words)
    #words = stemming(words)
    new.append(words)
  return new

In [0]:
X1_nn = preprocessing(X1)
X2_nn = preprocessing(X2)
X3_nn = preprocessing(X3)

In [0]:
len_list = [len(s) for s in X1_nn]
seq_length = max(len_list)

In [0]:
seq_length

29

#Add padding

In [0]:
def add_padding(corpus, seq_length):
    output = []
    for sentence in corpus:
        if len(sentence)>seq_length:
            output.append(sentence[:seq_length])
        else:
            for j in range(seq_length-len(sentence)):
                sentence.append("<PAD>")
            output.append(sentence)
    return output

text_train_pad = add_padding(X1_nn,seq_length)
text_dev_pad = add_padding(X2_nn,seq_length)
text_test_pad = add_padding(X3_nn,seq_length)

#Ebedding

In [0]:
import gensim.downloader as api
word_emb_model = api.load("glove-twitter-50")

[==================================================] 100.0% 199.5/199.5MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def get_embeddings(corpus,word_emb_model):
    emb_dim = word_emb_model.vector_size
    out = []
    for sentence in corpus:
        out_temp = []
        for word in sentence:
            try:
                out_temp.append(word_emb_model.wv[word])
            except:
                out_temp.append([0]*emb_dim)
    
        out.append(out_temp)
    return np.array(out)

train_emb = get_embeddings(text_train_pad,word_emb_model)
test_emb = get_embeddings(text_test_pad,word_emb_model)
dev_emb = get_embeddings(text_dev_pad,word_emb_model)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [0]:
def re_label(y):
  new_label = []
  for x in y:
    if x == '1':
      new_label.append("strongly_negative")
    elif x == '2':
      new_label.append("weakly_negative")
    elif x == '3':
      new_label.append("neutral")
    elif x == '4':
      new_label.append("weakly_positive")
    elif x == '5':
      new_label.append("strongly_positive")
  return new_label

y1_n = re_label(y1)
y2_n = re_label(y2)
y3_n = re_label(y3)

In [0]:
from sklearn.preprocessing import LabelEncoder
labels = np.unique(y1_n)

lEnc = LabelEncoder()
lEnc.fit(labels)
label_train_n = lEnc.transform(y1_n)
label_test_n = lEnc.transform(y3_n)
label_dev_n = lEnc.transform(y2_n)
numClass = len(labels)

print(labels)
print(lEnc.transform(labels))

['neutral' 'strongly_negative' 'strongly_positive' 'weakly_negative'
 'weakly_positive']
[0 1 2 3 4]


In [0]:
mr_class = list(labels)
mr_class

['neutral',
 'strongly_negative',
 'strongly_positive',
 'weakly_negative',
 'weakly_positive']

#LSTM

In [0]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(n_input, n_hidden, num_layers=2, batch_first =True, dropout = 0.1)
        self.linear = nn.Linear(n_hidden,n_class)

    def forward(self, x):
        x,_ = self.lstm(x)
        x = self.linear(x[:,-1,:])
        x = F.log_softmax(x, dim=1)
        return x

In [0]:
lstm_model = torch.load('/content/drive/My Drive/Anchor/SST_LSTM.pt')

In [0]:
test_torch = torch.from_numpy(np.array(test_emb)).float().to(device)
outtt = lstm_model(test_torch)
_, predicted = torch.max(outtt, 1)
y_pre=predicted.cpu().numpy()

from sklearn.metrics import classification_report
print(classification_report(label_test_n, y_pre))

              precision    recall  f1-score   support

           0       0.25      0.22      0.23       389
           1       0.35      0.33      0.34       279
           2       0.49      0.40      0.44       399
           3       0.42      0.45      0.44       633
           4       0.35      0.41      0.38       510

    accuracy                           0.38      2210
   macro avg       0.37      0.36      0.36      2210
weighted avg       0.38      0.38      0.38      2210



In [0]:
def predict_lr(texts):
  txt = preprocessing(texts)
  text_pad = add_padding(txt,seq_length)
  text_emb = get_embeddings(text_pad,word_emb_model)
  text_torch = torch.from_numpy(np.array(text_emb)).float().to(device)
  out = lstm_model(text_torch)
  _, predicted = torch.max(out, 1)
  y_pre=predicted.cpu().numpy()

  return y_pre

In [0]:
def findp(texts):
  txt = preprocessing(texts)
  text_pad = add_padding(txt,seq_length)
  text_emb = get_embeddings(text_pad,word_emb_model)
  text_torch = torch.from_numpy(np.array(text_emb)).float().to(device)
  out = lstm_model(text_torch)
  y_pre = out.data.cpu().numpy()
  y_exp = np.exp(y_pre)
  summ = 0
  for i in range(len(y_exp[0])):
      summ += y_exp[0][i]
  return y_exp/summ

In [0]:
true_index = []
for x in range(len(X3)):
  if y3_n[x] == labels[predict_lr([X3[x]])[0]]:
    true_index.append(x)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [0]:
len(true_index)

832

#Annotation

In [0]:
dataset = pytreebank.load_sst()
sample = dataset["test"][12]

In [0]:
for x in sample.to_labeled_lines():
  print(x)

(4, 'One of the greatest family-oriented , fantasy-adventure movies ever .')
(4, 'One of the greatest family-oriented , fantasy-adventure movies ever')
(4, 'One of the greatest family-oriented , fantasy-adventure movies')
(2, 'One')
(4, 'of the greatest family-oriented , fantasy-adventure movies')
(2, 'of')
(4, 'the greatest family-oriented , fantasy-adventure movies')
(4, 'the greatest family-oriented ,')
(4, 'the greatest family-oriented')
(2, 'the')
(3, 'greatest family-oriented')
(3, 'greatest')
(3, 'family-oriented')
(2, ',')
(2, 'fantasy-adventure movies')
(2, 'fantasy-adventure')
(2, 'movies')
(2, 'ever')
(2, '.')


#Anchor evaluation

In [0]:
pip install anchor_exp

     |████████████████████████████████| 430kB 4.8MB/s 
     |████████████████████████████████| 276kB 16.4MB/s 
     |████████████████████████████████| 2.0MB 20.6MB/s 
  Created wheel for anchor-exp: filename=anchor_exp-0.0.1.0-cp36-none-any.whl size=434742 sha256=210af094d24e8ebf1c598ab467578b8c86137ec13b7a186910c6d14ecc8ebb85
  Stored in directory: /root/.cache/pip/wheels/ff/b7/cd/7bfb36f4a01ff6b1509cd3432f8b208f455d232cee1079e309
  Created wheel for lime: filename=lime-0.2.0.0-cp36-none-any.whl size=284181 sha256=47debc4781c9c5e0b07bf0d59ab23ba366de34c5ecb2a56796deffb0cb70affc
  Stored in directory: /root/.cache/pip/wheels/22/f2/ec/e5ebd07348b2b1ac722e91c2f549fcc220f7d5f25497a61232
Successfully built anchor-exp lime
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0


In [0]:
pip install spacy && python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=73c9eff07f34bd0eeea3f1695850e6999b96f0b9d485c4d08704d584ab221dee
  Stored in directory: /tmp/pip-ephem-wheel-cache-_wmowlnj/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
import en_core_web_lg
from anchor import anchor_text
nlp = en_core_web_lg.load()

In [0]:
def anchor_evalution(data,index):
  evaluation = []
  for x in index:
    text = data[x]
    exp = explainer.explain_instance(text, predict_lr, threshold=0.99)
    sample = dataset["test"][x]
    min_sen = []
    min_len = seq_length
    for x in sample.to_labeled_lines():
      if x[0] ==  sample.to_labeled_lines()[0][0]:
        words = word_tokenize(x[1])
        if len(words) < min_len:
          min_len = len(words)
        min_sen.append(x[1])
    annotate = []
    for x in min_sen:
      if len(word_tokenize(x)) == min_len:
        annotate.append(x)
    annotation = preprocessing(annotate)
    match = 0
    for x in exp.names():
      if x in annotation[0]:
        match +=1
    if len(annotation[0]) == 0:
      evaluation_score = 0
    else:
      evaluation_score = (match)/(len(annotation[0]))
    evaluation.append(evaluation_score)
  return evaluation

In [0]:
explainer = anchor_text.AnchorText(nlp, mr_class, use_unk_distribution=True)

In [0]:
anchor_evalutions = anchor_evalution(X3,true_index[600:610])

In [0]:
print(true_index[600:610])
print(anchor_evalutions)

[1590, 1591, 1593, 1599, 1600, 1603, 1604, 1605, 1608, 1609]
[0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.8, 1.0, 1.0, 1.0]


In [0]:
overall = sum(anchor_evalutions)/len(anchor_evalutions)
print(overall)

0.6799999999999999


#Lime evaluation

In [0]:
sys.path.append('/content/drive/My Drive/Capstone/lime-master/lime')
import lime_text
from lime_text import LimeTextExplainer

In [0]:
def lime_evalution(data,index):
  evaluation = []
  for x in index:
    text = data[x]
    clss = label_test_n[x]
    sample = dataset["test"][x]
    min_sen = []
    min_len = seq_length
    for x in sample.to_labeled_lines():
      if x[0] ==  sample.to_labeled_lines()[0][0]:
        words = word_tokenize(x[1])
        if len(words) < min_len:
          min_len = len(words)
        min_sen.append(x[1])
    annotate = []
    for x in min_sen:
      if len(word_tokenize(x)) == min_len:
        annotate.append(x)
    annotation = preprocessing(annotate)

    if len(annotation[0]) == 0:
      exp = explainer.explain_instance(text,findp,num_features=1, labels=[clss,clss])
    else:
      exp = explainer.explain_instance(text,findp,num_features=len(annotation[0]), labels=[clss,clss])
    lime_explain = []
    for x in exp.as_list(clss):
      lime_explain.append(x[0])
    
    match = 0
    for x in lime_explain:
      if x in annotation[0]:
        match +=1
    if len(annotation[0]) == 0:
      evaluation_score = 0
    else:
      evaluation_score = (match)/(len(annotation[0]))
    evaluation.append(evaluation_score)
  return evaluation

In [0]:
explainer = LimeTextExplainer(class_names=mr_class)

In [0]:
lime_evalutions = lime_evalution(X3,true_index[600:610])

In [0]:
print(true_index[600:610])
print(lime_evalutions)

[1590, 1591, 1593, 1599, 1600, 1603, 1604, 1605, 1608, 1609]
[0.0, 0.0, 1.0, 0.0, 0.0, 0.3333333333333333, 0.4, 1.0, 0.0, 1.0]


In [0]:
overall = sum(lime_evalutions)/len(lime_evalutions)
print(overall)

0.37333333333333335


In [0]:
lime_evalutions = lime_evalution(X3,true_index[0:159])

/content/drive/My Drive/Capstone/lime-master/lime/lime_text.py:116: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/content/drive/My Drive/Capstone/lime-master/lime/lime_text.py:116: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/content/drive/My Drive/Capstone/lime-master/lime/lime_text.py:116: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `w

In [0]:
print(true_index[0:159])
print(lime_evalutions)
overall = sum(lime_evalutions)/len(lime_evalutions)
print(overall)


[4, 5, 6, 8, 9, 10, 11, 12, 19, 20, 23, 25, 26, 28, 30, 33, 37, 44, 46, 48, 50, 55, 57, 58, 59, 60, 61, 62, 65, 67, 72, 76, 82, 85, 86, 91, 92, 98, 99, 103, 106, 108, 109, 111, 114, 123, 129, 130, 131, 133, 134, 135, 138, 140, 143, 144, 147, 150, 151, 158, 160, 169, 170, 173, 175, 178, 179, 183, 185, 188, 189, 193, 194, 196, 197, 198, 199, 200, 201, 204, 209, 210, 211, 213, 216, 219, 222, 223, 224, 231, 234, 235, 236, 240, 241, 242, 243, 245, 249, 250, 253, 256, 260, 263, 264, 265, 266, 269, 270, 274, 277, 283, 286, 288, 291, 294, 298, 300, 302, 303, 304, 306, 307, 313, 314, 315, 321, 327, 333, 336, 338, 341, 343, 345, 346, 350, 359, 360, 361, 364, 365, 367, 370, 378, 385, 386, 390, 391, 394, 400, 401, 402, 403, 404, 406, 409, 411, 413, 416]
[0.8, 0.0, 0.0, 0.0, 1.0, 0.0, 0, 0.3333333333333333, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.8333333333333334, 0.0, 0.0, 0.0, 0.0, 0.6666666666666666, 0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.5, 0.0, 0.42857142857142855, 0.333